<a href="https://colab.research.google.com/github/Locomody/Automata_Theory/blob/main/Acceptor_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Implementation of acceptor model on python.

Acceptor is five-element tuple $A = (\Sigma,\ S,\ s_0,\  F \subset S\ , \delta: S \times \Sigma \rightarrow S)$ where:
* $\sigma$ - finite alphabet 
* $S$ - finite set of states 
* $s_0$ - the initial state
* $F \subset \mathcal{S}$ - finite set of final states
* $\delta: S \times \Sigma \rightarrow S$ - transition function

In [29]:
from typing import Set, Dict, Union, List


class Acceptor:

    def __init__(self,
                 alphabet: set,
                 states: set,
                 initial_state,
                 accepting_states: set,
                 transition_function: dict
                 ):
        
        if not isinstance(alphabet, set):
            for symbol in alphabet:
                if not self.__is_hashable(symbol):
                    raise TypeError(f"Symbol '{symbol}' of alphabet is not hashable.")
            alphabet = self.__make_alphabet(alphabet)
        

        if not isinstance(states, set):
            raise TypeError("States type is not set.")

        if initial_state not in states:
            raise ValueError("Initial states not in states.")

        if not isinstance(transition_function, dict):
            raise TypeError("Transition function type is not set.")
        
        if not isinstance(accepting_states, set):
            raise TypeError("Accepting states type is not set.")
        
        self.__alphabet = alphabet
        self.__states = states
        self.__initial_state = initial_state
        self.__accepting_states = accepting_states
        self.__transition_function = transition_function

    def accept(self, word: "Iterible") -> bool:
        """
        Check whether the acceptor accepts the word(iteratively).
        
        :param word: the list of symbols.
        :return: True if the acceptor accepts the word, False if not.
        """

        curr_state = self.__initial_state

        for symbol in word:

            if symbol not in self.__alphabet:
                return False

            elif self.__transition_function.get((curr_state, symbol)) is not None:
                curr_state = self.__transition_function.get((curr_state, symbol))

            else:
                return False
        return curr_state in self.__accepting_states
    
    def accept_reccursive(self, word: "Iterible") -> bool:
        """
        Check whether the acceptor accepts the word(recursively).

        :param word: the list of symbols.
        :return: True if the acceptor accepts the word, False if not.
        """
        def rec(word, curr_state):

            if len(word) == 0:
                return curr_state in self.__accepting_states
        
            else:
                curr_state = self.__transition_function[curr_state, word[0]]
                return rec(word[1::], curr_state)
        
        return rec(word, self._initial_state) 
    
    def __is_hashable(self, obj: "Object") -> bool:
        try:
            set([obj])
        except TypeError:
            return False
        return True

    
    def __make_alphabet(self, *args: "Iterible") -> set:
        """
        Returns set of symbols

        :return: Set that can be an alphabet
        """

        return set(args)
    


      

    @property
    def alphabet(self):
        return self.__alphabet
    
    @property
    def states(self):
        return self.__states
      
    @property
    def initial_state(self):
        return self.__initial_state
    
    @property
    def accepting_states(self):
        return self.__accepting_states
    
    @property
    def transition_function(self):
        return self.__transition_function


# Example of simple acceptor that accepts series of 1s and 0s if it is even number of 1s in series
alphabet = {"1", "0"}
states = {"even", "odd", "s3", "s4"}
initial_state = "even"
accepting_states = {"even"}
transition_function = {
    ("even", "1") : "odd",
    ("even", "0") : "even",
    ("odd", "1") : "even",
    ("odd", "0") : "odd"
}
even_ones = Acceptor(alphabet, states, initial_state, accepting_states, transition_function)
words = ["1101111", "1101011101", "100010101110101"]
tuple(even_ones.accept(word) for word in words)


(True, False, True)

#Delete unreachable states

The state $s$  of acceptor called unreachable if no string $w \in \Sigma^*$ for wich $p = \delta^*(s_0, w)$ 

Algorithm

1. Defined reachable states set as $\{s_0\}$ and new states set as $\{s_0\}$
2.  While new states not empty, we iterate throught all $ a \in \Sigma$ and add all $\delta(s, a)$ to new states set if its not in reacheble states set, and then add to reachable states set, for each $s$ in states set.
3. repeat 2
4. states = states - reachable_states 

In [30]:
def reachable(acceptor: Acceptor):
    """
    Delete all unreacheble states from acceptor

    :param acceptor: acceptor in wich we want to delete unreachable states
    :return: new acceptor without unreacheble states
    """
    reachable_states = {acceptor.initial_state}
    states = {acceptor.initial_state}

    while states:
        temp = {acceptor.transition_function[state, symbol] for state in states for symbol in acceptor.alphabet}
        states = temp - reachable_states
        reachable_states = reachable_states | states
    
    return Acceptor(acceptor.alphabet, reachable_states, acceptor.initial_state, acceptor.accepting_states, acceptor.transition_function)

print(even_ones.states)
even_ones = reachable(even_ones)
even_ones.states

{'s3', 'odd', 's4', 'even'}


{'even', 'odd'}

#Acceptor Reduction

Two states $s$ and $s'$ of acceptor $A$ called equivalent if exist no word that brings to differ states:

$\forall \sigma \in \Sigma^*.\ \delta^*(s, \sigma) = \delta^*(s', \sigma)$

In [32]:
import queue
def reduction(acceptor: Acceptor):
    def split(state_class, states, symbol):
        return {state_class_1 for state_class_1 in state_class
                if acceptor.transition_function[state_class_1, symbol] in states}, \
               {state_class_2 for state_class_2 in state_class
                if acceptor.transition_function[state_class_2, symbol] not in states}

    classes: List[Set[str]] = [acceptor.accepting_states, acceptor.states - acceptor.accepting_states]
    states_symbol_pairs_queue = queue.Queue()
    for symbol in acceptor.alphabet:
        states_symbol_pairs_queue.put((acceptor.accepting_states, symbol))
        states_symbol_pairs_queue.put((acceptor.states - acceptor.accepting_states, symbol))
    while not states_symbol_pairs_queue.empty():
        (states, symbol) = states_symbol_pairs_queue.get()
        for state_class in classes:
            state_class_1, state_class_2 = split(state_class, states, symbol)
            if state_class_1 and state_class_2:
                classes.remove(state_class)
                classes.append(state_class_1)
                classes.append(state_class_2)
                for token in acceptor.alphabet:
                    states_symbol_pairs_queue.put((state_class_1, token))
                    states_symbol_pairs_queue.put((state_class_2, token))
    new_states = {state_class.pop() for state_class in classes}
    return Acceptor(acceptor.alphabet, new_states, acceptor.initial_state, acceptor.accepting_states, acceptor.transition_function)
  
alphabet = {"1", "0"}
states = {"even", "odd", "odd2" "s3", "s4"}
initial_state = "even"
accepting_states = {"even"}
transition_function = {
    ("even", "0") : "even",
    ("even", "1") : "odd",
    ("odd", "0") : "odd2",
    ("odd", "1") : "even",
    ("odd2", "0") : "odd",
    ("odd2", "1") : "even"
}

acc = Acceptor(alphabet, states, initial_state, accepting_states, transition_function)
print(acc.states)
acc = reachable(acc)
print(acc.states)
acc = reduction(acc)
print(acc.states)

{'s4', 'odd', 'odd2s3', 'even'}
{'odd', 'odd2', 'even'}
{'odd', 'even'}
